In [1]:
!pip  install transformers==4.22.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.9 MB/s eta 0:00:00


In [2]:
!pip install autokeras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.9/944.9 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 50.3 MB/s eta 0:00:00


In [3]:
import transformers
import tensorflow as tf
import autokeras as ak

import numpy as np
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Using TensorFlow backend


In [4]:
deviceList = tf.test.gpu_device_name()
deviceList

'/device:GPU:0'

In [5]:
df = pd.read_csv("/content/labelled.csv")
df.head()

,Heading,Body,Category,URL
0,free speech not hate speech madras high court ...,madras high court issue significant remark ami...,Judiciary,https://www.indiatoday.in/law/high-courts/stor...
1,comment take context say us cop mock indian st...,seattle police officer guild friday come defen...,Crime,https://www.indiatoday.in/world/story/indian-s...
2,first meeting one nation one election committe...,first official meeting one nation one election...,Politics,https://www.indiatoday.in/india/story/one-nati...
3,us airlines flight depressurize midair plummet...,united airlines jet head rome turn around less...,Crime,https://www.indiatoday.in/world/story/us-fligh...
4,terrorist kill security force foil infiltratio...,three terrorist kill infiltration bid foil sec...,Crime,https://www.indiatoday.in/india/story/one-terr...


In [6]:
category_count = df['Category'].value_counts()

categories = category_count.index

categories


Index(['Entertainment', 'Business', 'Politics', 'Judiciary', 'Crime',
       'Culture', 'Sports', 'Science', 'International', 'Technology'],
      dtype='object')

In [7]:
categories = {
    "Entertainment" :   0,
"Business"  :       1,
"Politics"   :      2,
"Judiciary"   :     3,
"Crime"     :       4,
"Culture"    :      5,
"Sports"      :      6,
"Science"      :     7,
"International" :    8,
"Technology"     :   9
}

print(categories)


{'Entertainment': 0, 'Business': 1, 'Politics': 2, 'Judiciary': 3, 'Crime': 4, 'Culture': 5, 'Sports': 6, 'Science': 7, 'International': 8, 'Technology': 9}


In [8]:
def map_to_encoding(text):
    return categories.get(text, -1)  # Return -1 if not found

# Create the 'category_encoding' column
df['category_encoding'] = df['Category'].apply(map_to_encoding)

In [9]:
len(df)

11583

In [10]:
df.head()

,Heading,Body,Category,URL,category_encoding
0,free speech not hate speech madras high court ...,madras high court issue significant remark ami...,Judiciary,https://www.indiatoday.in/law/high-courts/stor...,3
1,comment take context say us cop mock indian st...,seattle police officer guild friday come defen...,Crime,https://www.indiatoday.in/world/story/indian-s...,4
2,first meeting one nation one election committe...,first official meeting one nation one election...,Politics,https://www.indiatoday.in/india/story/one-nati...,2
3,us airlines flight depressurize midair plummet...,united airlines jet head rome turn around less...,Crime,https://www.indiatoday.in/world/story/us-fligh...,4
4,terrorist kill security force foil infiltratio...,three terrorist kill infiltration bid foil sec...,Crime,https://www.indiatoday.in/india/story/one-terr...,4


In [11]:
newsArticlesBody = df['Body'].to_list()

categoryLabels = df['category_encoding'].to_list()

In [12]:
# Split the data into training, validation, and test sets (80% train, 10% val, 10% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    newsArticlesBody, categoryLabels, test_size=0.2, random_state=42, stratify = categoryLabels)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    test_texts, test_labels, test_size=0.5, random_state=42,  stratify = test_labels)

# Display the size of each dataset
print("Train set size:", len(train_texts))
print("Validation set size:", len(val_texts))
print("Test set size:", len(test_texts))

Train set size: 9266
Validation set size: 1158
Test set size: 1159


In [19]:
clf = ak.TextClassifier(max_trials=2, metrics=['accuracy'])  # Adjust max_trials as needed
# Convert text data to numpy arrays
X_train = np.array(train_texts)
y_train = np.array(train_labels)
X_val = np.array(val_texts)
y_val = np.array(val_labels)

# Train the model
clf.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
290/290 [==============================] - 43s 140ms/step - loss: 1.8750 - accuracy: 0.3352 - val_loss: 1.3030 - val_accuracy: 0.5639
Epoch 2/10
290/290 [==============================] - 3s 10ms/step - loss: 1.0539 - accuracy: 0.6688 - val_loss: 0.9530 - val_accuracy: 0.6926
Epoch 3/10
290/290 [==============================] - 3s 9ms/step - loss: 0.7262 - accuracy: 0.7778 - val_loss: 0.9464 - val_accuracy: 0.7098
Epoch 4/10
290/290 [==============================] - 3s 9ms/step - loss: 0.5321 - accuracy: 0.8373 - val_loss: 0.9999 - val_accuracy: 0.7193
Epoch 5/10
290/290 [==============================] - 4s 15ms/step - loss: 0.3836 - accuracy: 0.8825 - val_loss: 1.0884 - val_accuracy: 0.7150
Epoch 6/10
290/290 [==============================] - 3s 11ms/step - loss: 0.2837 - accuracy: 0.9140 - val_loss: 1.1978 - val_accuracy: 0.7124
Epoch 7/10
290/290 [==============================] - 3s 10ms/step - loss: 0.1899 - accuracy: 0.9392 - val_loss: 1.3822 - val_accuracy: 0.7073

In [20]:
test_texts = np.array(test_texts)
test_labels = np.array(test_labels)

# Now, you can proceed with evaluation
loss, accuracy = clf.evaluate(test_texts, test_labels)
print("Test accuracy:", accuracy)

37/37 [==============================] - 0s 6ms/step - loss: 1.6727 - accuracy: 0.7299
Test accuracy: 0.7299395799636841


In [56]:
keras_model = clf.export_model()
# Save the Keras model to an HDF5 file

In [60]:
print("Type of keras_model:", type(keras_model))


Type of keras_model: <class 'keras.src.engine.functional.Functional'>


In [62]:
# Save the model architecture as JSON
model_json = keras_model.to_json()
with open("AutoKerasModel.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights using TensorFlow format
keras_model.save("AutoKerasModel_weights", save_format="tf")


In [65]:
keras_model.save("temp1")

In [66]:
!zip -r /content/temp1.zip /content/temp1

  adding: content/temp1/ (stored 0%)
  adding: content/temp1/fingerprint.pb (stored 0%)
  adding: content/temp1/saved_model.pb (deflated 79%)
  adding: content/temp1/variables/ (stored 0%)
  adding: content/temp1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/temp1/variables/variables.index (deflated 50%)
  adding: content/temp1/keras_metadata.pb (deflated 88%)
  adding: content/temp1/assets/ (stored 0%)


In [45]:
keras_model.save("AutoKerasModel1", save_format='tf')

In [58]:
import pickle
# Save the model JSON to a pickle file
with open("AutoKerasModel_New.pkl", "wb") as file:
    pickle.dump(keras_model, file)


In [59]:
import joblib

# Save the entire model using joblib
with open("AutoKerasModel.joblib", "wb") as file:
    joblib.dump(keras_model, file)


In [38]:
test = keras_model.predict()

1/1 [==============================] - 0s 33ms/step


In [39]:
test[0].argmax()

0

In [40]:
def get_key_from_value(dictionary, value):
    # Iterate through the dictionary to find the key for the given value
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # Return None if the value is not found


In [41]:
value_to_find = test[0].argmax()
key_for_value = get_key_from_value(categories, value_to_find)

# Print the key for the given value
print("Key for value", value_to_find, "is:", key_for_value)

Key for value 0 is: Entertainment


In [46]:
loaded_model = tf.keras.models.load_model("AutoKerasModel1")


In [47]:
loaded_model.predict(["several report round yrfs tiger vs pathaan bring salman khan shah rukh khan together fullfledge film big deal hype possibility film create huge rumour claim aditya chopra go host joint narration tiger vs pathaan salman khan shah rukh khan yrf however say muchawaited film already narrate two superstar two separate meeting month backadvertisementsalman khan shah rukh khan give nod tiger vs pathaanthe script tiger vs pathaan already narrate salman khan shah rukh khan source also confirm aditya chopra narrate film megastar script lock team tiger vs pathaan start prep november shoot commence march next year yes film go floor march trade source say tiger vs pathaan script lock two megastar hindi cinema give nod script film huge milestone moment see two giant indian cinema come together fullfledge film boxoffice juggernaut karan arjun first love script get convince good satisfy expectation people big onscreen reunionthe person also add aditya chopra hold individual meeting srk salman respectively narrate film superstar love story film go floor marchmore aboit tiger vs pathaan yrf spy franchisethe script tiger vs pathaan already narrate salman khan shah rukh khan source also confirm aditya chopra narrate film megastar script lock team tiger vs pathaan start prep november shoot commence march next year yes film go floor march trade source say tiger vs pathaan script lock two megastar hindi cinema give nod script film huge milestone moment see two giant indian cinema come together fullfledge film boxoffice juggernaut karan arjun first love script get convince good satisfy expectation people big onscreen reunionthe person also add aditya chopra hold individual meeting srk salman respectively narrate film superstar love story film go floor marchmore aboit tiger vs pathaan yrf spy franchisedirecte siddharth anand tiger vs pathaan tout big film india ever produce part yrf spy universe kick tiger franchise star salman khan katrina kaif begin ek tha tiger tiger zinda hai continue war star hrithik roshan tiger shroffthen come pathaan star shah rukh khan deepika padukone john abraham alltime big hit hindi cinema four film yrf spy universe blockbuster next film spy franchise yrf tiger due release november diwali festival holiday"])

1/1 [==============================] - 0s 186ms/step


array([[9.9999905e-01, 3.0695049e-15, 3.3861314e-10, 2.5522180e-08,
        6.0001502e-09, 5.0353600e-07, 5.0163550e-07, 8.0316062e-15,
        2.1180309e-16, 1.0434401e-12]], dtype=float32)

In [52]:
!zip -r /content/AutoKerasZip.zip /content/AutoKerasModel1


  adding: content/AutoKerasModel1/ (stored 0%)
  adding: content/AutoKerasModel1/fingerprint.pb (stored 0%)
  adding: content/AutoKerasModel1/saved_model.pb (deflated 79%)
  adding: content/AutoKerasModel1/variables/ (stored 0%)
  adding: content/AutoKerasModel1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/AutoKerasModel1/variables/variables.index (deflated 50%)
  adding: content/AutoKerasModel1/keras_metadata.pb (deflated 88%)
  adding: content/AutoKerasModel1/assets/ (stored 0%)
